# 1. Simplex Method

In [1]:
c = [4, 5, 0, 0, 0, 0]
A = [
    [6, 4, 1, 0, 0, 0],
    [1, 2, 0, 1, 0, 0],
    [-1, 1, 0, 0, 1, 0],
    [0 , 1, 0, 0, 0, 1],
]
b = [24, 6, 1, 2]

import numpy as np
def to_tableau(c, A, b):
    xb = [eq + [x] for eq, x in zip(A, b)]
    z = c + [0]
    return xb + [z]
def can_be_improved(tableau):
    z = tableau[-1]
    return any(x > 0 for x in z[:-1])
import math

def get_pivot_position(tableau):
    z = tableau[-1]
    column = next(i for i, x in enumerate(z[:-1]) if x > 0)
    
    restrictions = []
    for eq in tableau[:-1]:
        el = eq[column]
        restrictions.append(math.inf if el <= 0 else eq[-1] / el)

    row = restrictions.index(min(restrictions))
    return row, column
def pivot_step(tableau, pivot_position):
    new_tableau = [[] for eq in tableau]
    
    i, j = pivot_position
    pivot_value = tableau[i][j]
    new_tableau[i] = np.array(tableau[i]) / pivot_value
    
    for eq_i, eq in enumerate(tableau):
        if eq_i != i:
            multiplier = np.array(new_tableau[i]) * tableau[eq_i][j]
            new_tableau[eq_i] = np.array(tableau[eq_i]) - multiplier
   
    return new_tableau
def is_basic(column):
    return sum(column) == 1 and len([c for c in column if c == 0]) == len(column) - 1

def get_solution(tableau):
    columns = np.array(tableau).T
    solutions = []
    for column in columns:
        solution = 0
        if is_basic(column):
            one_index = column.tolist().index(1)
            solution = columns[-1][one_index]
        solutions.append(solution)
        
    return solutions
def simplex(c, A, b):
    tableau = to_tableau(c, A, b)

    while can_be_improved(tableau):
        pivot_position = get_pivot_position(tableau)
        tableau = pivot_step(tableau, pivot_position)

    return get_solution(tableau)

solution = simplex(c, A, b)
print('solution: ', solution)

print("Solution = > ")
print(solution[0]*c[0] + solution[1]*c[1])

solution:  [3.0, 1.4999999999999998, 0, 0, 2.5000000000000004, 0.5000000000000002, 0]
Solution = > 
19.5


# 2. N/W Corner Method

In [13]:
def north_west_corner(supply, demand):
    supply_copy = supply.copy()
    demand_copy = demand.copy()
    i = 0
    j = 0
    bfs = []
    while len(bfs) < len(supply) + len(demand) - 1:
        s = supply_copy[i]
        d = demand_copy[j]
        v = min(s, d)
        supply_copy[i] -= v
        demand_copy[j] -= v
        bfs.append(((i, j), v))
        if supply_copy[i] == 0 and i < len(supply) - 1:
            i += 1
        elif demand_copy[j] == 0 and j < len(demand) - 1:
            j += 1
    return bfs
supply = [4, 5, 3, 9]
demand = [4, 4, 5, 3, 2, 3]
bfs = north_west_corner(supply, demand)
print(bfs)


[((0, 0), 4), ((1, 0), 0), ((1, 1), 4), ((1, 2), 1), ((2, 2), 3), ((3, 2), 1), ((3, 3), 3), ((3, 4), 2), ((3, 5), 3)]


In [21]:
m=int(input("Enter m:"))
n=int(input("Enter n:"))
print("Enter all costs in one line")
entries = list(map(int, input().split()))  
costs = np.array(entries).reshape(m, n)
print('\n The costs are:\n',costs)
print('\n')
supply = list(map(int,input("Enter the supply values : ").strip().split()))[:m]
print('\n')
demand = list(map(int,input("Enter the demand values : ").strip().split()))[:n]
print('\n')

def north_west_corner(supply, demand):
    supply_copy = supply.copy()
    demand_copy = demand.copy()
    i = 0
    j = 0
    bfs = []
    while len(bfs) < len(supply) + len(demand) - 1:
        s = supply_copy[i]
        d = demand_copy[j]
        v = min(s, d)
        supply_copy[i] -= v
        demand_copy[j] -= v
        bfs.append(((i, j), v))
        if supply_copy[i] == 0 and i < len(supply) - 1:
            i += 1
        elif demand_copy[j] == 0 and j < len(demand) - 1:
            j += 1
    return bfs

bfs = north_west_corner(supply, demand)
print(bfs)


Enter m:2
Enter n:3
Enter all costs in one line
1 2 3 4 5 6

 The costs are:
 [[1 2 3]
 [4 5 6]]


Enter the supply values : 1 2


Enter the demand values : 1 2


[((0, 0), 1), ((1, 0), 0), ((1, 1), 2)]


((1, 0), 0)

# 3. VAM Method

In [11]:
import numpy as np

def get_balanced(supply, demand, costs, penalties = None):
    total_supply = sum(supply)
    total_demand = sum(demand)
    
    if total_supply < total_demand:
        if penalties is None:
            raise Exception('Supply less than demand, penalties required')
        new_supply = supply + [total_demand - total_supply]
        new_costs = costs + [penalties]
        return new_supply, demand, new_costs
    if total_supply > total_demand:
        new_demand = demand + [total_supply - total_demand]
        new_costs = costs + [[0 for _ in demand]]
        return supply, new_demand, new_costs
    return supply, demand, costs

def vogel(supply, demand,costs):
    supply_copy = supply.copy()
    demand_copy = demand.copy()
    m=len(supply)
    n=len(demand)
    i = 0
    j = 0
    bfs = []
    while len(bfs) < len(supply) + len(demand) - 1:
        s = supply_copy[i]
        d = demand_copy[j]
        v = min(s, d)
        supply_copy[i] -= v
        demand_copy[j] -= v
        bfs.append(((i, j), v))
        if supply_copy[i] == 0 and i < len(supply) - 1:
            i += 1
        elif demand_copy[j] == 0 and j < len(demand) - 1:
            j += 1
    cost=0
    bfs_arr = [[0 for i in range(n)] for j in range(m)] 
    for item in bfs:
        bfs_arr[item[0][0]][item[0][1]]=item[1]
    print('\n The initial bfs is:\n',bfs_arr)
    for item in bfs:
        cost=cost+costs[item[0][0]][item[0][1]]*item[1]
    print('total bfs cost is: ',cost)
    return bfs

def get_us_and_vs(bfs, costs):
    us = [None] * len(costs)
    vs = [None] * len(costs[0])
    us[0] = 0
    bfs_copy = bfs.copy()
    while len(bfs_copy) > 0:
        for index, bv in enumerate(bfs_copy):
            i, j = bv[0]
            if us[i] is None and vs[j] is None: continue
                
            cost = costs[i][j]
            if us[i] is None:
                us[i] = cost - vs[j]
            else: 
                vs[j] = cost - us[i]
            bfs_copy.pop(index)
            break
            
    return us, vs

def get_ws(bfs, costs, us, vs):
    ws = []
    for i, row in enumerate(costs):
        for j, cost in enumerate(row):
            non_basic = all([p[0] != i or p[1] != j for p, v in bfs])
            if non_basic:
                ws.append(((i, j), us[i] + vs[j] - cost))
    
    return ws

def can_be_improved(ws):
    for p, v in ws:
        if v > 0: return True
    return False

def get_entering_variable_position(ws):
    ws_copy = ws.copy()
    ws_copy.sort(key=lambda w: w[1])
    return ws_copy[-1][0]

def get_possible_next_nodes(loop, not_visited):
    last_node = loop[-1]
    nodes_in_row = [n for n in not_visited if n[0] == last_node[0]]
    nodes_in_column = [n for n in not_visited if n[1] == last_node[1]]
    if len(loop) < 2:
        return nodes_in_row + nodes_in_column
    else:
        prev_node = loop[-2]
        row_move = prev_node[0] == last_node[0]
        if row_move: return nodes_in_column
        return nodes_in_row

def get_loop(bv_positions, ev_position):
    def inner(loop):
        if len(loop) > 3:
            can_be_closed = len(get_possible_next_nodes(loop, [ev_position])) == 1
            if can_be_closed: return loop
        
        not_visited = list(set(bv_positions) - set(loop))
        possible_next_nodes = get_possible_next_nodes(loop, not_visited)
        for next_node in possible_next_nodes:
            new_loop = inner(loop + [next_node])
            if new_loop: return new_loop
    
    return inner([ev_position])

def loop_pivoting(bfs, loop):
    even_cells = loop[0::2]
    odd_cells = loop[1::2]
    get_bv = lambda pos: next(v for p, v in bfs if p == pos)
    leaving_position = sorted(odd_cells, key=get_bv)[0]
    leaving_value = get_bv(leaving_position)
    
    new_bfs = []
    for p, v in [bv for bv in bfs if bv[0] != leaving_position] + [(loop[0], 0)]:
        if p in even_cells:
            v += leaving_value
        elif p in odd_cells:
            v -= leaving_value
        new_bfs.append((p, v))
        
    return new_bfs

def transportation_method(supply, demand, costs, penalties = None):
    balanced_supply, balanced_demand, balanced_costs = get_balanced(
        supply, demand, costs
    )
    def inner(bfs):
        us, vs = get_us_and_vs(bfs, balanced_costs)
        ws = get_ws(bfs, balanced_costs, us, vs)
        if can_be_improved(ws):
            ev_position = get_entering_variable_position(ws)
            loop = get_loop([p for p, v in bfs], ev_position)
            return inner(loop_pivoting(bfs, loop))
        return bfs
    
    basic_variables = inner(vogel(balanced_supply, balanced_demand,costs))
    ans = np.zeros((len(costs), len(costs[0])))
    for (i, j), v in basic_variables:
        ans[i][j] = int(v)

    return ans

def get_total_cost(costs, ans):
    total_cost = 0
    for i, row in enumerate(costs):
        for j, cost in enumerate(row):
            total_cost += cost * ans[i][j]
    return total_cost

m=int(input("Enter m:"))
n=int(input("Enter n:"))
print("Enter all costs in one line")
entries = list(map(int, input().split()))  
costs = np.array(entries).reshape(m, n)
print('\n The costs are:\n',costs)
print('\n')
supply = list(map(int,input("Enter the supply values : ").strip().split()))[:m]
print('\n')
demand = list(map(int,input("Enter the demand values : ").strip().split()))[:n]
print('\n')
ans = transportation_method(supply, demand, costs)
print('\n')
print('\n The optimal solution is:\n',ans)
print('total optimal cost: ', get_total_cost(costs, ans))

Enter m:4
Enter n:4
Enter all costs in one line
5 1 3 3 3 3 5 4 6 4 4 3 4 -1 4 2

 The costs are:
 [[ 5  1  3  3]
 [ 3  3  5  4]
 [ 6  4  4  3]
 [ 4 -1  4  2]]


Enter the supply values : 34 15 12 19


Enter the demand values : 21 25 17 17



 The initial bfs is:
 [[21, 13, 0, 0], [0, 12, 3, 0], [0, 0, 12, 0], [0, 0, 2, 17]]
total bfs cost is:  259



 The optimal solution is:
 [[ 6.  6. 17.  5.]
 [15.  0.  0.  0.]
 [ 0.  0.  0. 12.]
 [ 0. 19.  0.  0.]]
total optimal cost:  164.0


# 4. Disjkstra's Algorithm

In [2]:
import sys
vertices = [[0, 1, 1, 0, 0, 0, 0, 0, 0],
            [1, 0, 0, 1, 1, 0, 0, 0, 0],
            [1, 0, 0, 1, 0, 0, 1, 0, 0],
            [0, 1, 1, 0, 0, 1, 0, 0, 0],
            [0, 1, 0, 0, 0, 0, 0, 1, 0],
            [0, 0, 0, 1, 0, 0, 0, 1, 1],
            [0, 0, 1, 0, 0, 0, 0, 0, 1],
            [0, 0, 0, 0, 1, 1, 0, 0, 1],
            [0, 0, 0, 0, 0, 1, 1, 1, 0]]

edges = [[0, 7, 10, 0, 0, 0, 0, 0, 0],
            [7, 0, 0, 9, 27, 0, 0, 0, 0],
            [10, 0, 0, 8, 0, 0, 31, 0, 0],
            [0, 9, 8, 0, 0, 11, 0, 0, 0],
            [0, 27, 0, 0, 0, 0, 0, 15, 0],
            [0, 0, 0, 11, 0, 0, 0, 15, 17],
            [0, 0, 31, 0, 0, 0, 0, 0, 32],
            [0, 0, 0, 0, 15, 15, 0, 0, 21],
            [0, 0, 0, 0, 0, 17, 32, 21, 0]]

def to_be_visited():
    global visited_and_distance
    v = -10
    for index in range(num_of_vertices):
        if visited_and_distance[index][0] == 0 \
            and (v < 0 or visited_and_distance[index][1] <=
                 visited_and_distance[v][1]):
            v = index
    return v


num_of_vertices = len(vertices[0])

visited_and_distance = [[0, 0]]
for i in range(num_of_vertices-1):
    visited_and_distance.append([0, sys.maxsize])

for vertex in range(num_of_vertices):

    to_visit = to_be_visited()
    for neighbor_index in range(num_of_vertices):

        if vertices[to_visit][neighbor_index] == 1 and \
                visited_and_distance[neighbor_index][0] == 0:
            new_distance = visited_and_distance[to_visit][1] \
                + edges[to_visit][neighbor_index]
            if visited_and_distance[neighbor_index][1] > new_distance:
                visited_and_distance[neighbor_index][1] = new_distance
        
        visited_and_distance[to_visit][0] = 1

i = 0
j = 0

for distance in visited_and_distance:
    
    print("Distance of ", chr(ord('a') + i),
          " from source vertex: ", distance[1])
    
    i = i + 1
    j = j + i

j = j - 1
    
print(j)    

Distance of  a  from source vertex:  0
Distance of  b  from source vertex:  7
Distance of  c  from source vertex:  10
Distance of  d  from source vertex:  16
Distance of  e  from source vertex:  34
Distance of  f  from source vertex:  27
Distance of  g  from source vertex:  41
Distance of  h  from source vertex:  42
Distance of  i  from source vertex:  44
44


# 5. Floyd Algorithm

In [3]:
nV = 5
INF = 999

def floyd_warshall(G):
    distance = list(map(lambda i: list(map(lambda j: j, i)), G))

    for k in range(nV):
        for i in range(nV):
            for j in range(nV):
                distance[i][j] = min(distance[i][j], distance[i][k] + distance[k][j])
    print_solution(distance)

def print_solution(distance):
    for i in range(nV):
        for j in range(nV):
            if(distance[i][j] == INF):
                print("INF", end=" ")
            else:
                print(distance[i][j], end="  ")
        print(" ")


G = [[0, 3, 10, INF, INF],
         [3, 0, INF, 5, INF],
         [10, INF, 0, 6, 15],
         [INF, 5, 6, 0, 4],
         [INF, INF, INF, 4, 0]]

floyd_warshall(G)

0  3  10  8  12   
3  0  11  5  9   
10  11  0  6  10   
8  5  6  0  4   
12  9  10  4  0   


In [ ]:
def initialise(V):
    global dis, Next
    for i in range(V):
        for j in range(V):
            dis[i][j] = graph[i][j]
            if (graph[i][j] == INF):
                Next[i][j] = -1
            else:
                Next[i][j] = j
                
def constructPath(u, v):
    global graph, Next    
    if (Next[u][v] == -1):
        return {}
    path = [u]
    while (u != v):
        u = Next[u][v]
        path.append(u)
    return path

def floydWarshall(V):
    global dist, Next
    for k in range(V):
        for i in range(V):
            for j in range(V):
                if (dis[i][k] == INF or dis[k][j] == INF):
                    continue
                if (dis[i][j] > dis[i][k] + dis[k][j]):
                    dis[i][j] = dis[i][k] + dis[k][j]
                    Next[i][j] = Next[i][k]
                    
def printPath(path):
    n = len(path)
    for i in range(n - 1):
        print(path[i], end=" -> ")
    print (path[n - 1])
    
if __name__ == '__main__':
    MAXM,INF = 100,10**7
    dis = [[-1 for i in range(MAXM)] for i in range(MAXM)]
    Next = [[-1 for i in range(MAXM)] for i in range(MAXM)]
    V = 4
    graph = [ [ 0, 3, INF, 7 ],
            [ 8, 0, 2, INF ],
            [ 5, INF, 0, 1 ],
            [ 2, INF, INF, 0 ] ]
    
    initialise(V)
    floydWarshall(V)
    path = []

    print("Shortest path from 1 to 3: ", end = "")
    path = constructPath(1, 3)
    printPath(path)
 
    # Path from node 0 to 2
    print("Shortest path from 0 to 2: ", end = "")
    path = constructPath(0, 2)
    printPath(path)
 
    # Path from node 3 to 2
    print("Shortest path from 3 to 2: ", end = "")
    path = constructPath(3, 2)
    printPath(path)

# 6. Prim's Algorithm

In [4]:
INF = 9999999
V = 5
G = [[0, 9, 75, 0, 0],
     [9, 0, 95, 19, 42],
     [75, 95, 0, 51, 66],
     [0, 19, 51, 0, 31],
     [0, 42, 66, 31, 0]]
selected = [0, 0, 0, 0, 0]
no_edge = 0
selected[0] = True
print("Edge : Weight\n")
while (no_edge < V - 1):
    minimum = INF
    x = 0
    y = 0
    for i in range(V):
        if selected[i]:
            for j in range(V):
                if ((not selected[j]) and G[i][j]):  
                    if minimum > G[i][j]:
                        minimum = G[i][j]
                        x = i
                        y = j
    print(str(x) + "-" + str(y) + ":" + str(G[x][y]))
    selected[y] = True
    no_edge += 1

Edge : Weight

0-1:9
1-3:19
3-4:31
3-2:51


# 7. Kruskal's Algorithm

In [5]:
class Graph:
    def __init__(self, vertices):
        self.V = vertices
        self.graph = []

    def add_edge(self, u, v, w):
        self.graph.append([u, v, w])

    def find(self, parent, i):
        if parent[i] == i:
            return i
        return self.find(parent, parent[i])

    def apply_union(self, parent, rank, x, y):
        xroot = self.find(parent, x)
        yroot = self.find(parent, y)
        if rank[xroot] < rank[yroot]:
            parent[xroot] = yroot
        elif rank[xroot] > rank[yroot]:
            parent[yroot] = xroot
        else:
            parent[yroot] = xroot
            rank[xroot] += 1

    def kruskal_algo(self):
        result = []
        i, e = 0, 0
        self.graph = sorted(self.graph, key=lambda item: item[2])
        parent = []
        rank = []
        for node in range(self.V):
            parent.append(node)
            rank.append(0)
        while e < self.V - 1:
            u, v, w = self.graph[i]
            i = i + 1
            x = self.find(parent, u)
            y = self.find(parent, v)
            if x != y:
                e = e + 1
                result.append([u, v, w])
                self.apply_union(parent, rank, x, y)
        for u, v, weight in result:
            print("%d - %d: %d" % (u, v, weight))


g = Graph(6)
g.add_edge(0, 1, 4)
g.add_edge(0, 2, 4)
g.add_edge(1, 2, 2)
g.add_edge(1, 0, 4)
g.add_edge(2, 0, 4)
g.add_edge(2, 1, 2)
g.add_edge(2, 3, 3)
g.add_edge(2, 5, 2)
g.add_edge(2, 4, 4)
g.add_edge(3, 2, 3)
g.add_edge(3, 4, 3)
g.add_edge(4, 2, 4)
g.add_edge(4, 3, 3)
g.add_edge(5, 2, 2)
g.add_edge(5, 4, 3)
g.kruskal_algo()

1 - 2: 2
2 - 5: 2
2 - 3: 3
3 - 4: 3
0 - 1: 4


# 8. CPM

In [19]:
import pandas as pd
import datetime

import matplotlib.pyplot as plt
import networkx as nx
from criticalpath import Node

# set up the tasks:
tasks = [("A", {"Duration": 2}), 
         ("B", {"Duration": 4}), 
         ("C", {"Duration": 3}), 
         ("D", {"Duration": 1}), 
         ("E", {"Duration": 6}),
         ("F", {"Duration": 5}),
         ("G", {"Duration": 7}),
         ("H", {"Duration": 2})]

dependencies = [("A", "D"), 
                ("B", "E"), 
                ("C", "F"),
                ("D", "G"), 
                ("E", "G"),
                ("G", "H"),
                ("F", "H")]

proj = Node('Project')
for t in tasks:
    proj.add(Node(t[0], duration=t[1]["Duration"]))
for d in dependencies:
    proj.link(d[0],d[1])
proj.update_all()
crit_path = [str(n) for n in proj.get_critical_path()]
proj_duration = proj.duration

print(f"The current critical path is: {crit_path}")
print("-"*50)
print(f"The value of the CPM is: {proj_duration} days")

The current critical path is: ['B', 'E', 'G', 'H']
--------------------------------------------------
The value of the CPM is: 19 days


# 9. PERT

In [6]:
import math
import logging

logging.basicConfig(filename='logger.log', level=logging.INFO, format='%(name)s - %(levelname)s - %(message)s')

class Activity:
    def __init__(self, name, duration):
        logging.info("new activity has been created, name: " + name + ", duration: " + str(duration))
        self.name = str(name).lower()
        self.duration = duration

    # str method for activity
    def __str__(self):
        logging.info("str method called for '" + self.name + "' activity")
        return "Activity Name: " + self.name + ", Duration: " + str(self.duration)


    def __repr__(self):
        logging.info("repr method called for '" + self.name + "' activity")
        return self.name

class Pert:
    def __init__(self, graph={}):
        logging.info("new pert: '" + repr(self) + "' has been created, graph: " + str(graph))
        self.forward_dict = graph   
        self.backward_dict = {}     
        self.info_dict = {}         
        self.start_activity = Activity
        self.end_activity = Activity
        self.reset_initial_graph()  
        self.generate_info()       

    def __str__(self):
        logging.info("str method called for '" + repr(self) + "' pert")
        iterator = iter(self)
        graph_str = 'Activities:\n'
        for activity in iterator:
            graph_str += str(activity) + '\n'
        return (graph_str + 'Connections:\n' 
            + str(self.forward_dict)
            + '\nProject Duration:\n' 
            + str(self.info_dict[self.end_activity]['ef']))

    def __iter__(self):
        logging.info("new iterator for '" + repr(self) + "' pert")
        return iter(self.forward_dict)

    def reset_initial_graph(self):
        logging.info("'reset_initial_graph' method has been called for '" + repr(self) + "' pert")
        for activity in self.forward_dict:
            self.backward_dict[activity] = []
        for activity in self.forward_dict:
            if activity.name == "start":
                self.start_activity = activity
            if activity.name == "end":
                self.end_activity = activity
            for node in self.forward_dict[activity]:
                self.backward_dict[node].append(activity)
        self.reset_info()

    def reset_info(self):
        logging.info("'reset_info' method has been called for '" + repr(self) + "' pert")
        for activity in self.forward_dict:
            self.info_dict[activity] = {
                "duration": activity.duration, 
                "es": 0, "ef": 0, "ls": 0, "lf": math.inf, 
                "slack": 0}

    def generate_info(self):
        logging.info("'generate_info' method has been called for '" + repr(self) + "' pert")
        if self.forward_dict == {}:
            return
        self.info_dict[self.start_activity]["ef"] = self.info_dict[self.start_activity]["duration"]
        self.start_to_end_scan(self.start_activity)
        self.info_dict[self.end_activity]["lf"] = self.info_dict[self.end_activity]["ef"]
        self.info_dict[self.end_activity]["ls"] = self.info_dict[self.end_activity]["lf"] - self.info_dict[self.end_activity]["duration"]
        self.end_to_start_scan(self.end_activity)
        self.clatulate_slack()
        self.generate_info_for_isolated()

    def start_to_end_scan(self, activity):
        logging.info("'start_to_end_scan' method has been called for '" + repr(self) + "' pert")
        for node in self.forward_dict[activity]:
            if self.info_dict[activity]["ef"] > self.info_dict[node]["es"]:
                self.info_dict[node]["es"] = self.info_dict[activity]["ef"]
                self.info_dict[node]["ef"] = self.info_dict[node]["es"] + self.info_dict[node]["duration"]
            self.start_to_end_scan(node)

    def end_to_start_scan(self, activity):
        logging.info("'end_to_start_scan' method has been called for '" + repr(self) + "' pert")
        for node in self.backward_dict[activity]:
            if (self.info_dict[node]["lf"] > self.info_dict[activity]["ls"]):
                self.info_dict[node]["lf"] = self.info_dict[activity]["ls"]
                self.info_dict[node]["ls"] = (self.info_dict[node]["lf"] - self.info_dict[node]["duration"])
            self.end_to_start_scan(node)

    def clatulate_slack(self):
        logging.info("'clatulate_slack' method has been called for '" + repr(self) + "' pert")
        for activity in self.forward_dict:
            self.info_dict[activity]["slack"] = self.info_dict[activity]["lf"] - self.info_dict[activity]["ef"]

    def generate_info_for_isolated(self):
        logging.info("'generate_info_for_isolated' method has been called for '" + repr(self) + "' pert")
        isolated = self.find_isolated()
        for activity in isolated:
            self.info_dict[activity]["ef"] = self.info_dict[activity]["es"] + self.info_dict[activity]["duration"]
            self.info_dict[activity]["lf"] = self.info_dict[self.end_activity]["lf"]
            self.info_dict[activity]["ls"] = self.info_dict[activity]["lf"] - self.info_dict[activity]["duration"]
            self.info_dict[activity]["slack"] = self.info_dict[activity]["lf"] - self.info_dict[activity]["ef"]

    def add_activity(self, activity, in_connections=[], out_connections=[]):
        logging.info("'add_activity' method has been called for '" + repr(self) + "' pert")
        if activity in self.forward_dict:
            return
        self.forward_dict[activity] = out_connections
        self.backward_dict[activity] = in_connections
        if in_connections != []:
            for node in in_connections:
                if self.forward_dict[node] is None:
                    self.forward_dict[node] = []
                self.forward_dict[node] += [activity]
        if out_connections != []:
            for node in out_connections:
                if self.backward_dict[node] is None:
                    self.backward_dict[node] = []
                self.backward_dict[node] += [activity]
        self.info_dict[activity] = {
            "duration": activity.duration, 
            "es": 0, "ef": 0, "ls": 0, "lf": math.inf, 
            "slack": 0}
        self.reset_info()
        self.generate_info()

    def find_isolated(self):
        logging.info("'find_isolated' method has been called for '" + repr(self) + "' pert")
        isolated = list(self.info_dict)
        for activity in self.forward_dict:
            if self.forward_dict[activity] != [] and activity in isolated:
                isolated.remove(activity)
        for activity in self.backward_dict:
            if self.backward_dict[activity] != [] and activity in isolated:
                isolated.remove(activity)
        return isolated
    
    def get_slack_for_each_activity(self):
        logging.info("'get_slack_for_each_activity' method has been called for '" + repr(self) + "' pert")
        slacks = {activity: self.info_dict[activity]["slack"] for activity in self.info_dict if self.info_dict[activity]["slack"] != 0}
        return sorted(slacks.items(), key=lambda kv: kv[1], reverse=True)
    
    def get_sum_of_slacks(self):
        logging.info("'get_sum_of_slacks' method has been called for '" + repr(self) + "' pert")
        slacks = [kv[1] for kv in self.get_slack_for_each_activity()]
        return sum(slacks)

    def get_critical_path(self):
        logging.info("'get_critical_path' method has been called for '" + repr(self) + "' pert")
        activity = self.start_activity
        path = [activity]
        while activity != self.end_activity :
            for node in self.forward_dict[activity]:
                if self.info_dict[node]["slack"] == 0:
                    activity = node
            path += [activity]
        return path

    def get_critical_path_with_length(self):
        logging.info("'get_critical_path_with_length' method has been called for '" + repr(self) + "' pert")
        return {activity: activity.duration for activity in self.get_critical_path()}
    
    def shorten_critical_path(self):
        logging.info("'shorten_critical_path' method has been called for '" + repr(self) + "' pert")
        critical_path = self.get_critical_path()
        max_decrease_to_activities = {activity: activity.duration - 1 for activity in critical_path}
        for i in range(0,  len(critical_path), 1):
            for j in range(2, len(critical_path) - i, 1):
                for path in self.get_all_alternative_paths(critical_path[i], critical_path[i + j]):
                    for activity in critical_path[i + 1 : i + j : 1]:
                        if path[1] not in critical_path and max_decrease_to_activities[activity] >= self.info_dict[path[1]]["slack"]:
                            max_decrease_to_activities[activity] = self.info_dict[path[1]]["slack"] - 1
        return max_decrease_to_activities

    def get_all_alternative_paths(self, start_activity, end_activity, path=[]):
        logging.info("'get_all_alternative_paths' method has been called for '" + repr(self) + "' pert")
        one_path = path + [start_activity]
        if start_activity == end_activity:
            return [one_path]
        if start_activity not in self.info_dict:
            return []
        paths = []
        for activity in self.forward_dict[start_activity]:
            paths += self.get_all_alternative_paths(activity, end_activity, one_path)
        return paths

if __name__ == "__main__":
    start = Activity("start", 5)
    a = Activity("a", 2)
    b = Activity("b", 3)
    c = Activity("c", 3)
    d = Activity("d", 4)
    e = Activity("e", 3)
    f = Activity("f", 6)
    end = Activity("end", 2)
    graph = {start: [a, d, f], a: [b], b: [c], c: [end], d: [e], e: [end], f:[end], end:[]}
    
    print("initialize a graph:")
    pert = Pert(graph)
    
    j = Activity("j", 16)
    print("add activity to project:")
    pert.add_activity(j, [start], [end])
    
    print("print activity:")
    print(j)
    print("critical path:")
    print(pert.get_critical_path())
    
    print("maximum shorting times:")
    print(pert.shorten_critical_path())
    
    print("slack time in descending order:")
    print(pert.get_slack_for_each_activity())
    
    print("sum of slack times:")
    print(pert.get_sum_of_slacks())
    
    print("iterate over all the activities with iterator:")
    for activity in iter(pert):
        print(activity)
        
    print("isolated activities:")
    print(pert.find_isolated())
    print("print pert:")
    print(pert)

initialize a graph:
add activity to project:
print activity:
Activity Name: j, Duration: 16
critical path:
[start, j, end]
maximum shorting times:
{start: 4, j: 7, end: 1}
slack time in descending order:
[(f, 10), (d, 9), (e, 9), (a, 8), (b, 8), (c, 8)]
sum of slack times:
52
iterate over all the activities with iterator:
Activity Name: start, Duration: 5
Activity Name: a, Duration: 2
Activity Name: b, Duration: 3
Activity Name: c, Duration: 3
Activity Name: d, Duration: 4
Activity Name: e, Duration: 3
Activity Name: f, Duration: 6
Activity Name: end, Duration: 2
Activity Name: j, Duration: 16
isolated activities:
[]
print pert:
Activities:
Activity Name: start, Duration: 5
Activity Name: a, Duration: 2
Activity Name: b, Duration: 3
Activity Name: c, Duration: 3
Activity Name: d, Duration: 4
Activity Name: e, Duration: 3
Activity Name: f, Duration: 6
Activity Name: end, Duration: 2
Activity Name: j, Duration: 16
Connections:
{start: [a, d, f, j], a: [b], b: [c], c: [end], d: [e], e: [e

# 10. Assignment Problem

In [10]:
from ortools.linear_solver import pywraplp


def main():
    costs = [
        [90, 80, 75, 70],
        [35, 85, 55, 65],
        [125, 95, 90, 95],
        [45, 110, 95, 115],
        [50, 100, 90, 100],
    ]
    num_workers = len(costs)
    num_tasks = len(costs[0])
    solver = pywraplp.Solver.CreateSolver('SCIP')
    x = {}
    for i in range(num_workers):
        for j in range(num_tasks):
            x[i, j] = solver.IntVar(0, 1, '')

    for i in range(num_workers):
        solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

    for j in range(num_tasks):
        solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)

    objective_terms = []
    for i in range(num_workers):
        for j in range(num_tasks):
            objective_terms.append(costs[i][j] * x[i, j])
    solver.Minimize(solver.Sum(objective_terms))

    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        print(f'Total cost = {solver.Objective().Value()}\n')
        for i in range(num_workers):
            for j in range(num_tasks):
                if x[i, j].solution_value() > 0.5:
                    print(f'Worker {i} assigned to task {j}.' +
                          f' Cost: {costs[i][j]}')
    else:
        print('No solution found.')


if __name__ == '__main__':
    main()

Total cost = 265.0

Worker 0 assigned to task 3. Cost: 70
Worker 1 assigned to task 2. Cost: 55
Worker 2 assigned to task 1. Cost: 95
Worker 3 assigned to task 0. Cost: 45
